# Advent of Code 2022: Day 21
https://adventofcode.com/2022/day/21


## Part 1
Calculate the value of the 'root' equation, which recursively requires other equations to be calculated.

### Get the data into a list of strings

In [201]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Function to preprocess the data into a dictionary

In [202]:
def preprocess_data(data):
  output_dict = {}
  for item in data:
    
    split = item.split(': ')
    
    # If the second item in the split
    # is a digit, then create an entry
    # with the first item as key and
    # the second item, converted to 
    # integer, as value. 
    if split[1].isdigit():
      output_dict[split[0]] = int(split[1])
    
    # Otherwise the second item should
    # consist of three parts; two keys
    # and an operator. Make these into
    # a list and make an entry with
    # the first item as key and the
    # list as value.
    else:
      split2 = split[1].split(' ')
      output_dict[split[0]] = split2

  return output_dict

In [203]:
data_dict = preprocess_data(data_list)

### Function to find the number at a given key

In [204]:
def get_number(data_dict, key):
  item = data_dict[key]
  
  # If the item is an int,
  # return it.
  if isinstance(item, int):
    return item
  
  # Otherwise, the item is a list of two keys and
  # and an operator. Call the function recursively
  # using the elements in the list.
  else:
    if item[1] == '+':
      return get_number(data_dict, item[0]) + get_number(data_dict, item[2])
    elif item[1] == '-':
      return get_number(data_dict, item[0]) - get_number(data_dict, item[2])
    elif item[1] == '*':
      return get_number(data_dict, item[0]) * get_number(data_dict, item[2])
    elif item[1] == '/':
      return get_number(data_dict, item[0]) // get_number(data_dict, item[2])

In [205]:
get_number(data_dict, 'root')

155708040358220

### 

## Part 2


### Function to get all the numbers at and before a given key

In [206]:
def get_all_numbers(data_dict, key, path_dict):
  item = data_dict[key]
  
  # If the item is an int,
  # return it and the
  # path dictionary.
  if isinstance(item, int):
    path_dict[key] = item
    return item, path_dict
  
  # Otherwise, the item is a list of two keys and
  # and an operator. Call the function recursively
  # using the elements in the list. Keep updating
  # the path dictionary. 
  else:
    if item[1] == '+':
      n1, path_dict = get_all_numbers(data_dict, item[0],path_dict)
      n2, path_dict = get_all_numbers(data_dict, item[2],path_dict)
      path_dict[key] = n1+n2
      return n1+n2, path_dict
    elif item[1] == '-':
      n1, path_dict = get_all_numbers(data_dict, item[0],path_dict)
      n2, path_dict = get_all_numbers(data_dict, item[2],path_dict)
      path_dict[key] = n1-n2
      return n1-n2, path_dict
    elif item[1] == '*':
      n1, path_dict = get_all_numbers(data_dict, item[0],path_dict)
      n2, path_dict = get_all_numbers(data_dict, item[2],path_dict)
      path_dict[key] = n1*n2
      return n1*n2, path_dict
    elif item[1] == '/':
      n1, path_dict = get_all_numbers(data_dict, item[0],path_dict)
      n2, path_dict = get_all_numbers(data_dict, item[2],path_dict)
      path_dict[key] = n1/n2
      return n1/n2, path_dict

### Function to get the path between two keys

In [207]:
def get_path(data_dict, start, goal='root'):
  path = []
  curr = start
  while curr != goal:
    
    # Go through the dictionary and find the
    # key which has curr in its list.
    # Append curr to the path list and set
    # curr to the found key. Iterate until
    # reaching the goal.
    for key in data_dict:
      if isinstance(data_dict[key], list):
        if curr in data_dict[key]:
          path.append(curr)
          curr = key
  
  # Return the path in
  # reverse order.
  path.reverse()
  return path

In [208]:
def change_start(data_dict, start, goal='root'):
  
  # Get the dictionary of all the values
  # up to the goal key, as well as the 
  # path from the start to the goal.
  path_dict = {}
  n, path_dict = get_all_numbers(data_dict, goal, path_dict)
  path = get_path(data_dict, start, goal)

  # Set curr to the list of the
  # first key in the path list, which
  # should be directly before the goal.
  # Then set the goal to be the value of
  # the key in 'goal' which is not in the
  # path list. 
  curr = data_dict[path[0]]
  if path[0] == data_dict[goal][0]:
    goal = path_dict[data_dict[goal][2]]
  else:
    goal = path_dict[data_dict[goal][0]]

  for i in range(1, len(path)):
    
    # If element 0 in curr is in
    # the path list, this item needs
    # to change. If its not in the 
    # list it should be kept fixed.
    # Also keep track of this order 
    # for the minus operator.
    if curr[0] in path:
      change = path_dict[curr[0]]
      order = 0
      fixed = path_dict[curr[2]]
    else: 
      change = path_dict[curr[2]]
      order = 1
      fixed = path_dict[curr[0]]
    
    # Change the value depending
    # on the operator. 
    if curr[1] == '+':
      change = goal-fixed
    elif curr[1] == '-':
      if order == 0:
        change = goal+fixed
      elif order ==1:
        change = -(goal-fixed)
    elif curr[1] == '*':
      change = goal/fixed
    elif curr[1] == '/':
      change = goal*fixed

    # Update the current to the next
    # element in the path list, and
    # update the goal to the changed
    # value.
    curr = data_dict[path[i]]
    goal = change
  return int(goal)

In [209]:
change_start(data_dict, 'humn')

3342154812537